## Main
> Main class for Linkedin_AI

In [ ]:
#| default_exp main

In [ ]:
#| export

import json
import os
import asyncio
from typing import List
import aiofiles
from openai import AsyncOpenAI
from mlflow import trace

from linkedin_ai.document import Document
from linkedin_ai.retrievers import BM25Retriever, VectorRetriever, BaseRetriever



class LinkedinAI:
    """Main RAG system for LinkedIn posts with factory methods for different retrieval strategies."""
    
    def __init__(
        self, 
        retriever: BaseRetriever,
        model: str = "gpt-4o",
        max_tokens: int = 1000,
        temperature: float = 0.1,
        verbose: bool = False
    ):
        self.retriever = retriever
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.client = AsyncOpenAI()
    
    @classmethod
    async def _load_posts(cls, file_path: str) -> List[Document]:
        """Load LinkedIn posts from a JSON file."""
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Data file {file_path} not found")
        
        documents = []
        try:
            async with aiofiles.open(file_path, mode='r', encoding='utf-8') as file:
                content = await file.read()
                posts_data = json.loads(content)
                
            for post_id, post_info in posts_data.items():
                doc = Document(
                    id=post_id,
                    content=post_info.get("content", ""),
                    url=post_info.get("url", ""),
                    date=post_info.get("date", "unknown")
                )
                documents.append(doc)
                
            print(f"Loaded {len(documents)} LinkedIn posts")
            return documents
        except Exception as e:
            raise Exception(f"Error loading data: {e}")
    
    @classmethod
    async def from_bm25(
        cls,
        posts: str,
        model: str = "gpt-4o",
        top_k: int = 3,
        max_tokens: int = 1000,
        temperature: float = 0.1,
        verbose: bool = False
    ) -> "LinkedinAI":
        """Create a LinkedinAI instance using BM25 retrieval."""

        # Load documents
        documents = await cls._load_posts(posts)
        
        # Create and initialize retriever
        retriever = BM25Retriever(documents, top_k)
        await retriever.initialize()
        
        return cls(retriever,model, max_tokens, temperature, verbose)
    
    @classmethod
    async def from_vector_search(
        cls,
        posts: str,
        embedding_model: str = "text-embedding-ada-002",
        model: str = "gpt-4o",
        top_k: int = 3,
        max_tokens: int = 1000,
        temperature: float = 0.1,
        verbose: bool = False
    ) -> "LinkedinAI":
        """Create a LinkedinAI instance using vector search retrieval."""
        # Initialize OpenAI client with Instructor
        
        # Load documents
        documents = await cls._load_posts(posts)
        
        # Create vector file path
        vector_file = posts.replace(".json", "_vectors.npy")
        
        # Create and initialize retriever
        retriever = VectorRetriever(embedding_model, documents, top_k)
        # Pass the client during initialization
        await retriever.initialize(vector_file)
        
        return cls(retriever, model, max_tokens, temperature, verbose)
    
    @trace()
    async def ask(self, query: str, verbose: bool = False) -> str:
        """Answer a query using the RAG system."""
        if verbose:
            print(f"Query: {query}")
        
        # Retrieve relevant documents
        relevant_docs = await self.retriever.retrieve(query)
        
        if verbose:
            print(f"Retrieved {len(relevant_docs)} documents")
            for i, doc in enumerate(relevant_docs):
                print(f"Document {i+1}: {doc}")
        
        # Format context from retrieved documents
        context = "\n\n".join([
            f"Post {i+1} (Date: {doc.date}):\n{doc.content}\nURL: {doc.url}"
            for i, doc in enumerate(relevant_docs)
        ])
        
        # Generate answer using instructor and Pydantic model
        system_message = "You are a helpful assistant that provides information based on given context. Do not add any additional information not present in the context."
        user_message = f"Here is the context:\n\n{context}\n\nBased on this information, please answer: {query}"
        
        response = await self.client.chat.completions.create(
            model=self.model,
            max_tokens=self.max_tokens,
            temperature=self.temperature,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ]
        )
        
        
        return response.choices[0].message.content.strip()


###  Example usage


In [ ]:
posts = {
  "7316949721615405057": {
    "content": "Memory Networks and its descendants deserve more attention (so to speak 😂).",
    "url": "https://www.linkedin.com/posts/yann-lecun_at-the-10th-year-anniversary-of-our-memory-activity-7316949721615405057-GdGw?utm_source=share&utm_medium=member_desktop&rcm=ACoAACHEQ1kBau5gFVkfSgsSB2flft8HtbfWS74",
    "date": "1h"
  },
  "7316885912183857152": {
    "content": "Just six weeks in Meta working on Llama’s multimodal capabilities, and it’s been all about moving fast. I have already contributed to Llama4’s image grounding capabilities.\n\nLlama4 Scout is the best-in-class on image grounding, able to align user prompts with relevant visual concepts and anchor model responses to regions in the image. This enables more precise visual question answering for the LLM to better understand user intent and localize objects of interest. \n\nI am super thrilled that the grounding capabilities got featured as a demo in the Llama blog post (link below)! Many thanks to the awesome teammates for the opportunities and the support.\n\nLink to LLAMA-4: https://www.llama.com/",
    "url": "https://www.linkedin.com/posts/yash-patel-93626945_just-six-weeks-in-meta-working-on-llamas-ugcPost-7314834583294787584-srbb?utm_source=share&utm_medium=member_desktop&rcm=ACoAACHEQ1kBau5gFVkfSgsSB2flft8HtbfWS74",
    "date": "5d"
  },
  "7316885428165373952": {
    "content": "On April 3, the VinFuture Foundation successfully orchestrated the final \"Call for Nominations\" webinar in preparation for the 2025 Award season. It meticulously provided comprehensive guidelines for nomination partners and scientists interested in the Prize. The webinar was attended by over 120 scientists and experts from 26 countries worldwide.\n\nThe webinar was chaired by the esteemed innovator Professor Thuc-Quyen Nguyen, Chair of the VinFuture Prize Pre-Screening Committee. Co-chairing the session was Professor Martin Andrew Green, a Member of the VinFuture Prize Council and Laureate of the 2023 VinFuture Prize. Together, they disseminated crucial insights regarding the nomination procedures and encouraged robust participation from the scientific community. Notably, the presence of Professor Yann LeCun, a 2024 VinFuture Grand Prize Laureate recognized for his significant contributions to the AI revolution, served as a profound source of inspiration for researchers and nomination partners to embrace and honor future technological advancements.",
    "url": "https://www.linkedin.com/posts/vinfuture-prize_on-april-3-the-vinfuture-foundation-successfully-ugcPost-7315269860001755138-qtJo?utm_source=share&utm_medium=member_desktop&rcm=ACoAACHEQ1kBau5gFVkfSgsSB2flft8HtbfWS74",
    "date": "4d"
  },
}

with open("posts.json", "w") as f:
    json.dump(posts, f)

In [ ]:

my_rag = await LinkedinAI.from_bm25(
    posts="posts.json",
    model="gpt-4o",
    top_k=3,
    verbose=True
)

await my_rag.ask("What are your thoughts on RAG?")



# Example with Vector Search
print("\n=== Using Vector Search retrieval ===")
my_rag = await LinkedinAI.from_vector_search(
    posts="posts.json",
    embedding_model="text-embedding-ada-002",
    model="gpt-4o",
    top_k=3,
    verbose=True
)

await my_rag.ask("How do you evaluate AI systems?")

    

Loaded 3 LinkedIn posts
BM25 index initialized

=== Using Vector Search retrieval ===
Loaded 3 LinkedIn posts


"The context provided does not contain specific information on how to evaluate AI systems. It includes details about a webinar for the VinFuture Prize, work on Llama4's image grounding capabilities, and a mention of Memory Networks, but it does not discuss evaluation methods for AI systems."